In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words =  open('names.txt').read().splitlines()
words[-8:]

['yama',
 'yami',
 'yamuna',
 'yash',
 'yashoda',
 'yuda',
 'yudhishthira',
 'yudhisthira']

In [3]:
len(words)

32616

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [103]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
    
    # print(w)
    context = [0]*block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join([itos[i] for i in context]), '--->', itos[ix])
        context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [98]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([232331, 3]), torch.int64, torch.Size([232331]), torch.int64)

In [99]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [20,  8,  9],
        [ 8,  9, 18],
        [ 9, 18,  1]])

In [100]:
Y

tensor([ 5, 13, 13,  ..., 18,  1,  0])

In [101]:
C = torch.randn((27, 2))

In [102]:
C[5]

tensor([1.9085, 0.2503])

In [70]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [71]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [72]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[-0.7849, -0.9960,  0.7573,  ...,  0.9980, -0.4400, -1.0000],
        [-0.9020, -0.9984,  0.3193,  ...,  0.9661, -0.5037, -1.0000],
        [-0.9056, -0.8885,  0.8583,  ...,  0.9319, -0.9971, -1.0000],
        ...,
        [-0.9549, -0.9594,  0.7838,  ..., -0.8140, -0.9988, -0.9985],
        [-0.9993,  0.8194, -0.9192,  ..., -0.9995,  0.8908,  0.6793],
        [-0.8250,  0.9799, -0.7458,  ..., -0.9918, -0.9841,  0.9736]])

In [73]:
h.shape

torch.Size([32, 100])

In [74]:
(emb.view(-1, 6) @ W1).shape

torch.Size([32, 100])

In [75]:
b1.shape

torch.Size([100])

In [76]:
# 32, 100
#  1, 100

In [77]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [78]:
logits = h @ W2 + b2

In [79]:
logits.shape

torch.Size([32, 27])

In [80]:
counts = logits.exp()

In [81]:
prob = counts / counts.sum(-1, keepdim=True)

In [82]:
prob.shape

torch.Size([32, 27])

In [83]:
prob[0].sum()

tensor(1.0000)

In [84]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(21.7791)

In [85]:
## ----------------Let's make it in a sense :) -------------------

In [104]:
X.shape, Y.shape  # dataset

(torch.Size([232331, 3]), torch.Size([232331]))

In [105]:
g = torch.Generator().manual_seed(2147483647)  # reproducibility
C = torch.randn((27, 2), generator=g)  # embedding
W1 = torch.randn((6, 100), generator=g)  # weights
b1 = torch.randn(100, generator=g)  # bias
W2 = torch.randn((100, 27), generator=g)  # weights
b2 = torch.randn(27, generator=g)  # bias
parameters = [C, W1, b1, W2, b2]

In [106]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [107]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)  # (32, 100)
logits = h @ W2 + b2  # (32, 27)
# counts = logits.exp()
# prob = counts / counts.sum(1, keepdim=True)
# loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Y)  # same as the above but more stable and efficient (above is for raw understanding)
loss

tensor(19.4980)

In [108]:
logits =torch.tensor([-100, -3, 0, 100])
counts = logits.exp()
probs = counts / counts.sum()
probs

tensor([0., 0., 0., nan])

In [109]:
counts

tensor([3.7835e-44, 4.9787e-02, 1.0000e+00,        inf])

In [110]:
# That's why we should use the cross-entropy loss function

In [111]:
## ------------------- Training -------------------

In [112]:
for p in parameters:
    p.requires_grad = True

In [122]:
for i in range(10):

    # minibatch construction
    ix = torch.randint(0, X.shape[0], (32,))
    # forward pass
    emb = C[X[ix]]                                 # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)  # (32, 100)
    logits = h @ W2 + b2                       # (32, 27)
    loss = F.cross_entropy(logits, Y[ix])        
    # print(i, loss.item())

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    for p in parameters:
        p.data += -0.1 * p.grad

print(loss.item())

2.670947790145874


In [115]:
torch.randint(0, X.shape[0], (32,))

tensor([108755,  69866,  30770, 226971,  99304,  88290, 185606, 104171,  72512,
         28270, 135818, 131037, 193014,  88121, 114482, 190931, 132862,  26906,
        117039,  25329,  29714,   3435,  44776,  28473, 115657, 164749, 211277,
        177809,  52285, 152048,  35533, 173526])